<a href="https://colab.research.google.com/github/Chaitya0623/Infiheal_Codes/blob/main/Infiheal_ner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
dict={'color':{'primary':['red','yellow','blue'],'secondary':['orange','green','violet']},1:'abc',2.0:'pqr'}
print(dict.keys())
print(dict.values())
dict['color']['secondary'][1]

dict_keys(['color', 1, 2.0])
dict_values([{'primary': ['red', 'yellow', 'blue'], 'secondary': ['orange', 'green', 'violet']}, 'abc', 'pqr'])


'green'

In [ ]:
import pandas as pd

df1 = pd.read_csv('/content/Indian-Female-Names.csv')
df1.drop(columns=['gender', 'race'], inplace=True)
df1.rename(columns = {'name':'text'}, inplace = True)
df1['text'] = df1['text'].str.split().str[0]
df2 = pd.read_csv('/content/Indian-Male-Names.csv')
df2.drop(columns=['gender', 'race'], inplace=True)
df2.rename(columns = {'name':'text'}, inplace = True)
df2['text'] = df2['text'].str.split().str[0]
df3 = pd.read_csv('/content/Indian_Names.csv')
df3.drop(columns=['Unnamed: 0'], inplace=True)
df3.rename(columns = {'Name':'text'}, inplace = True)
df4 = pd.read_csv('/content/first_name.txt', header=None, names=['text'])
df = pd.concat([df1, df2, df3, df4], ignore_index=True)

In [ ]:
df = df.drop_duplicates()
df = df.dropna().reset_index(drop=True)
df

,text
0,shivani
1,isha
2,smt
3,divya
4,mansi
...,...
8996,Zoheb
8997,Zoraiz
8998,Zoya
8999,Zufeshan


In [ ]:
df.to_csv('indian_names.csv')

Creating Model


In [ ]:
import pandas as pd
df = pd.read_csv('/content/indian_names.csv')
df.drop(columns=['Unnamed: 0'], inplace=True)
df

,text
0,shivani
1,isha
2,smt
3,divya
4,mansi
...,...
8996,Zoheb
8997,Zoraiz
8998,Zoya
8999,Zufeshan


In [ ]:
import string
def remove_punctuation(text):
    # Create a translation table to remove punctuation
    translator = str.maketrans('', '', string.punctuation)
    # Remove punctuation using the translation table
    return text.translate(translator)
df = df.applymap(remove_punctuation)
df

,text
0,shivani
1,isha
2,smt
3,divya
4,mansi
...,...
8996,Zoheb
8997,Zoraiz
8998,Zoya
8999,Zufeshan


In [ ]:
def has_hindi(text):
    # Regular expression to identify Hindi characters
    hindi_pattern = re.compile(r'[ऀ-ॿ]')
    # Check if the text contains any Hindi characters
    return hindi_pattern.search(text) is not None
df = df[~df['text'].apply(has_hindi)]
df = df.reset_index(drop=True)

In [ ]:
# Function to remove numbers using regular expression
def remove_numbers(text):
    return re.sub(r'\d+', '', text)

# Remove numbers from the 'Text' column
df['text'] = df['text'].apply(remove_numbers)
df = df.reset_index(drop=True)

In [ ]:
df['annotation'] = ''
for i in range(len(df)):
  df['annotation'][i] = [(df['text'][i],'PERSON')]
df

,text,annotation
0,shivani,"[(shivani, PERSON)]"
1,isha,"[(isha, PERSON)]"
2,smt,"[(smt, PERSON)]"
3,divya,"[(divya, PERSON)]"
4,mansi,"[(mansi, PERSON)]"
...,...,...
8915,Zoheb,"[(Zoheb, PERSON)]"
8916,Zoraiz,"[(Zoraiz, PERSON)]"
8917,Zoya,"[(Zoya, PERSON)]"
8918,Zufeshan,"[(Zufeshan, PERSON)]"


In [ ]:
import numpy as np
df.replace('', np.nan, inplace=True)
df.dropna(inplace=True)
df

,text,annotation
0,shivani,"[(shivani, PERSON)]"
1,isha,"[(isha, PERSON)]"
2,smt,"[(smt, PERSON)]"
3,divya,"[(divya, PERSON)]"
4,mansi,"[(mansi, PERSON)]"
...,...,...
8915,Zoheb,"[(Zoheb, PERSON)]"
8916,Zoraiz,"[(Zoraiz, PERSON)]"
8917,Zoya,"[(Zoya, PERSON)]"
8918,Zufeshan,"[(Zufeshan, PERSON)]"


In [ ]:
df.to_csv('fuckyoubro.csv')

In [ ]:
from sklearn.model_selection import train_test_split

train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)
test_df, dev_df = train_test_split(temp_df, test_size=0.5, random_state=42)

In [ ]:
train_df = train_df.dropna().reset_index(drop=True)
train_df

,text,annotation
0,Janhvi,"[(Janhvi, PERSON)]"
1,goldi,"[(goldi, PERSON)]"
2,nrendr,"[(nrendr, PERSON)]"
3,trijugi,"[(trijugi, PERSON)]"
4,prabhatyashi,"[(prabhatyashi, PERSON)]"
...,...,...
6238,zafar,"[(zafar, PERSON)]"
6239,nischaya,"[(nischaya, PERSON)]"
6240,moham,"[(moham, PERSON)]"
6241,mohseena,"[(mohseena, PERSON)]"


In [ ]:
import pandas as pd
from tqdm import tqdm
from difflib import SequenceMatcher
import re
import pickle

def matcher(string, pattern):
    '''
    Return the start and end index of any pattern present in the text.
    '''
    match_list = []
    pattern = pattern.strip()
    seqMatch = SequenceMatcher(None, string, pattern, autojunk=False)
    match = seqMatch.find_longest_match(0, len(string), 0, len(pattern))
    if (match.size == len(pattern)):
        start = match.a
        end = match.a + match.size
        match_tup = (start, end)
        string = string.replace(pattern, "X" * len(pattern), 1)
        match_list.append(match_tup)

    return match_list, string

def mark_sentence(s, match_list):
    '''
    Marks all the entities in the sentence as per the BIO scheme.
    '''
    word_dict = {}
    for word in s.split():
        word_dict[word] = 'O'

    for start, end, e_type in match_list:
        temp_str = s[start:end]
        tmp_list = temp_str.split()
        if len(tmp_list) > 1:
            word_dict[tmp_list[0]] = 'B-' + e_type
            for w in tmp_list[1:]:
                word_dict[w] = 'I-' + e_type
        else:
            word_dict[temp_str] = 'B-' + e_type
    return word_dict

def clean(text):
    '''
    Just a helper fuction to add a space before the punctuations for better tokenization
    '''
    filters = ["!", "#", "$", "%", "&", "(", ")", "/", "*", ".", ":", ";", "<", "=", ">", "?", "@", "[",
               "\\", "]", "_", "`", "{", "}", "~", "'"]
    for i in text:
        if i in filters:
            text = text.replace(i, " " + i)

    return text

def create_data(df, filepath):
    '''
    The function responsible for the creation of data in the said format.
    '''
    with open(filepath , 'w') as f:
        for text, annotation in zip(df.text, df.annotation):
            text = clean(text)
            text_ = text
            match_list = []
            for i in annotation:
                a, text_ = matcher(text, i[0])
                match_list.append((a[0][0], a[0][1], i[1]))

            d = mark_sentence(text, match_list)

            for i in d.keys():
                f.writelines(i + ' ' + d[i] +'\n')
            f.writelines('\n')

def main():
    ## path to save the txt file.
    filepath = 'data/train.txt'
    ## creating the file.
    create_data(train_df, filepath)

if __name__ == '__main__':
    main()

In [ ]:
import pandas as pd
from tqdm import tqdm
from difflib import SequenceMatcher
import re
import pickle

def matcher(string, pattern):
    '''
    Return the start and end index of any pattern present in the text.
    '''
    match_list = []
    pattern = pattern.strip()
    seqMatch = SequenceMatcher(None, string, pattern, autojunk=False)
    match = seqMatch.find_longest_match(0, len(string), 0, len(pattern))
    if (match.size == len(pattern)):
        start = match.a
        end = match.a + match.size
        match_tup = (start, end)
        string = string.replace(pattern, "X" * len(pattern), 1)
        match_list.append(match_tup)

    return match_list, string

def mark_sentence(s, match_list):
    '''
    Marks all the entities in the sentence as per the BIO scheme.
    '''
    word_dict = {}
    for word in s.split():
        word_dict[word] = 'O'

    for start, end, e_type in match_list:
        temp_str = s[start:end]
        tmp_list = temp_str.split()
        if len(tmp_list) > 1:
            word_dict[tmp_list[0]] = 'B-' + e_type
            for w in tmp_list[1:]:
                word_dict[w] = 'I-' + e_type
        else:
            word_dict[temp_str] = 'B-' + e_type
    return word_dict

def clean(text):
    '''
    Just a helper fuction to add a space before the punctuations for better tokenization
    '''
    filters = ["!", "#", "$", "%", "&", "(", ")", "/", "*", ".", ":", ";", "<", "=", ">", "?", "@", "[",
               "\\", "]", "_", "`", "{", "}", "~", "'"]
    for i in text:
        if i in filters:
            text = text.replace(i, " " + i)

    return text

def create_data(df, filepath):
    '''
    The function responsible for the creation of data in the said format.
    '''
    with open(filepath , 'w') as f:
        for text, annotation in zip(df.text, df.annotation):
            text = clean(text)
            text_ = text
            match_list = []
            for i in annotation:
                a, text_ = matcher(text, i[0])
                match_list.append((a[0][0], a[0][1], i[1]))

            d = mark_sentence(text, match_list)

            for i in d.keys():
                f.writelines(i + ' ' + d[i] +'\n')
            f.writelines('\n')

def main():
    ## path to save the txt file.
    filepath = 'data/test.txt'
    ## creating the file.
    create_data(test_df, filepath)

if __name__ == '__main__':
    main()

In [ ]:
import pandas as pd
from tqdm import tqdm
from difflib import SequenceMatcher
import re
import pickle

def matcher(string, pattern):
    '''
    Return the start and end index of any pattern present in the text.
    '''
    match_list = []
    pattern = pattern.strip()
    seqMatch = SequenceMatcher(None, string, pattern, autojunk=False)
    match = seqMatch.find_longest_match(0, len(string), 0, len(pattern))
    if (match.size == len(pattern)):
        start = match.a
        end = match.a + match.size
        match_tup = (start, end)
        string = string.replace(pattern, "X" * len(pattern), 1)
        match_list.append(match_tup)

    return match_list, string

def mark_sentence(s, match_list):
    '''
    Marks all the entities in the sentence as per the BIO scheme.
    '''
    word_dict = {}
    for word in s.split():
        word_dict[word] = 'O'

    for start, end, e_type in match_list:
        temp_str = s[start:end]
        tmp_list = temp_str.split()
        if len(tmp_list) > 1:
            word_dict[tmp_list[0]] = 'B-' + e_type
            for w in tmp_list[1:]:
                word_dict[w] = 'I-' + e_type
        else:
            word_dict[temp_str] = 'B-' + e_type
    return word_dict

def clean(text):
    '''
    Just a helper fuction to add a space before the punctuations for better tokenization
    '''
    filters = ["!", "#", "$", "%", "&", "(", ")", "/", "*", ".", ":", ";", "<", "=", ">", "?", "@", "[",
               "\\", "]", "_", "`", "{", "}", "~", "'"]
    for i in text:
        if i in filters:
            text = text.replace(i, " " + i)

    return text

def create_data(df, filepath):
    '''
    The function responsible for the creation of data in the said format.
    '''
    with open(filepath , 'w') as f:
        for text, annotation in zip(df.text, df.annotation):
            text = clean(text)
            text_ = text
            match_list = []
            for i in annotation:
                a, text_ = matcher(text, i[0])
                match_list.append((a[0][0], a[0][1], i[1]))

            d = mark_sentence(text, match_list)

            for i in d.keys():
                f.writelines(i + ' ' + d[i] +'\n')
            f.writelines('\n')

def main():
    ## path to save the txt file.
    filepath = 'data/dev.txt'
    ## creating the file.
    create_data(dev_df, filepath)

if __name__ == '__main__':
    main()

In [ ]:
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 54.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 91.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 59.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 54.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB

In [ ]:
from flair.datasets import ColumnCorpus
from flair.data import Corpus
# define columns
columns = {0 : 'text', 1 : 'ner'}
# directory where the data resides
data_folder = '/content/data'
# initializing the corpus
corpus: Corpus = ColumnCorpus(data_folder, columns,
                              train_file = 'train.txt',
                              test_file = 'test.txt',
                              dev_file = 'dev.txt')

2023-08-03 12:50:25,422 Reading data from /content/data
2023-08-03 12:50:25,424 Train: /content/data/train.txt
2023-08-03 12:50:25,427 Dev: /content/data/dev.txt
2023-08-03 12:50:25,428 Test: /content/data/test.txt


In [ ]:
import torch

corpus = corpus

# 2. what tag do we want to predict?
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)

# 4. initialize fine-tuneable transformer embeddings WITH document context
from flair.embeddings import TransformerWordEmbeddings

embeddings = TransformerWordEmbeddings(
    model='xlm-roberta-large',
    layers="-1",
    subtoken_pooling="first",
    fine_tune=True,
    use_context=True,
)

# 5. initialize bare-bones sequence tagger (no CRF, no RNN, no reprojection)
from flair.models import SequenceTagger

tagger = SequenceTagger(
    hidden_size=256,
    embeddings=embeddings,
    tag_dictionary=tag_dictionary,
    tag_type='ner',
    use_crf=False,
    use_rnn=False,
    reproject_embeddings=False,
)

# 6. initialize trainer with AdamW optimizer
from flair.trainers import ModelTrainer

trainer = ModelTrainer(tagger, corpus)

# 7. run training with XLM parameters (20 epochs, small LR)
from torch.optim.lr_scheduler import OneCycleLR

trainer.fine_tune('resources/taggers/ner-english-large',
              learning_rate=5.0e-6,
              mini_batch_size=4,
              mini_batch_chunk_size=1,
              max_epochs=20,
              scheduler=OneCycleLR,
              embeddings_storage_mode='none',
              weight_decay=0.,
              )

<ipython-input-21-c7b154118e18>:9: DeprecationWarning: Call to deprecated method make_tag_dictionary. (Use 'make_label_dictionary' instead.) -- Deprecated since version 0.8.
  tag_dictionary = corpus.make_tag_dictionary(tag_type=tag_type)


2023-08-03 12:51:09,523 SequenceTagger predicts: Dictionary with 3 tags: O, <START>, <STOP>
2023-08-03 12:51:09,535 ----------------------------------------------------------------------------------------------------
2023-08-03 12:51:09,539 Model: "SequenceTagger(
  (embeddings): TransformerWordEmbeddings(
    (model): XLMRobertaModel(
      (embeddings): XLMRobertaEmbeddings(
        (word_embeddings): Embedding(250003, 1024)
        (position_embeddings): Embedding(514, 1024, padding_idx=1)
        (token_type_embeddings): Embedding(1, 1024)
        (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): XLMRobertaEncoder(
        (layer): ModuleList(
          (0-23): 24 x XLMRobertaLayer(
            (attention): XLMRobertaAttention(
              (self): XLMRobertaSelfAttention(
                (query): Linear(in_features=1024, out_features=1024, bias=True)
                (key): Linear(in_featur

100%|██████████| 335/335 [01:18<00:00,  4.24it/s]

2023-08-03 12:58:50,415 Evaluating as a multi-label problem: False
2023-08-03 12:58:50,435 0.0	0.0	0.0	0.0
2023-08-03 12:58:50,436 
Results:
- F-score (micro) 0.0
- F-score (macro) 0.0
- Accuracy 0.0

By class:
              precision    recall  f1-score   support

      PERSON     0.0000    0.0000    0.0000    1338.0

   micro avg     0.0000    0.0000    0.0000    1338.0
   macro avg     0.0000    0.0000    0.0000    1338.0
weighted avg     0.0000    0.0000    0.0000    1338.0

2023-08-03 12:58:50,437 ----------------------------------------------------------------------------------------------------


{'test_score': 0.0,
 'dev_score_history': [],
 'train_loss_history': [],
 'dev_loss_history': []}